<a href="https://colab.research.google.com/github/TuckerArrants/nlp/blob/master/Text_Generation_with_HuggingFace_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

# Experimenting with HuggingFace - Text Generation

*Author: Tucker Arrants*

**I have recently decided to explore the ins and outs of the 😊 Transformers library and this is the next chapter in that journey. In this notebook, I will explore text generation using a GPT-2, which was trained the predict next words on 40GB of Internet text data. The fully trained model is actually not available as the creators were concerned about '[malicious applications of the technology](https://openai.com/blog/better-language-models/)', but there is a much smaller version that is available for enthusiants to play with, which we will use here**

**In this notebook, we will explore different decoding methods like Beam search, Top-K sampling, and Top-P sampling, demonstrating their performance along the way. This project is a work in progress and I will continually update it as I learn more about text generation. Feel free to comment with any questions/suggestions:**

In [2]:
#for reproducability
SEED = 50

#maximum number of words in output text
MAX_LEN = 34

# I. Intro

**A language model is a machine learning model that can look at part of a sentence and predict the next word/sequence of words. Much like the autofill features on your iPhone/Android, GPT-2 is capable of next word prediction on a much larger and more sophisticated scale. For reference, the smallest available GPT-2 has 117 million parameters, whereas the largest one (invisible to the public) has over 1.5 billion parameters. The largest one available for public use is half the size of their main GPT-2 model**

**😊 Transformers makes it very easy to import this model with both PyTorch and TensorFlow - in this notebook we will be using TensorFlow but it is just as easy in PyTorch. Both the model and its Tokenizer can be imported from the `transformers` library that anyone can get by typing `!pip install transformers`. Let's see just how simple it is to generate text with a neural network. We begin with our input sequence:**

In [3]:
input_sequence = "Liverpool is the best soccer team in the Premier League"

**We will choose the largest available GPT-2 model but it is easy to install the other sizes if you want to mess around with them:**

In [4]:
#get transformers
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

#get large GPT2 tokenizer and GPT2 model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)

#tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
#GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-medium", pad_token_id=tokenizer.eos_token_id)

#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
#GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

#view model parameters
GPT2.summary()

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2-large.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgp_t2lm_head_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
transformer (TFGPT2MainLayer multiple                  774030080 
Total params: 774,030,080
Trainable params: 774,030,080
Non-trainable params: 0
_________________________________________________________________


**Wow, we just imported a deep learning model with more than 774 million parameters in just a couple lines of code with HuggingFace. Now let's see what we can do with it:**

# II. Different Decoding Methods

## First Pass (Greedy Search)

**With Greedy search, the word with the highest probability is predicted as the next word i.e. the next word is updated via:**

$$w_t = argmax_{w}P(w | w_{1:t-1})$$

**at each timestep $t$. Let's see how this naive approach performs:**

In [5]:
#get deep learning basics
import tensorflow as tf
tf.random.set_seed(SEED)

In [6]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(input_sequence, return_tensors='tf')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = GPT2.generate(input_ids, max_length = MAX_LEN)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
Liverpool is the best soccer team in the Premier League, and they're going to be the best team in the Premier League for a long time," he said. "They


**And there we go: generating text is that easy. Our results are not great - as we can see, our model starts repeating itself rather quickly. The main issue with Greedy Search is that words with high probabilities can be masked by words in front of them with low probabilities, so the model is unable to explore more diverse combinations of words. We can prevent this by implementing Beam Search:**

## Beam Search with N-Gram Penalities

**Beam search is essentially Greedy Search but the model tracks and keeps `num_beams` of hypotheses at each time step, so the model is able to compare alternative paths as it generates text. We can also include a n-gram penalty by setting `no_repeat_ngram_size = 2` which ensures that no 2-grams appear twice. We will also set `num_return_sequences = 5` so we can see what the other 5 beams looked like**

**To use Beam Search, we need only modify some parameters in the `generate` function:**

In [7]:
# set return_num_sequences > 1
beam_outputs = GPT2.generate(
    input_ids, 
    max_length = MAX_LEN, 
    num_beams = 5, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 5, 
    early_stopping = True
)

print('')
print("Output:\n" + 100 * '-')

# now we have 3 output sequences
for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


Output:
----------------------------------------------------------------------------------------------------
0: Liverpool is the best soccer team in the Premier League," he said.

"It's a great club and I'm very proud to be a part of it.
1: Liverpool is the best soccer team in the Premier League," he said.

"It's a great club and I'm very proud to be a part of it."
2: Liverpool is the best soccer team in the Premier League," he said.

"It's a great club and I'm very proud to be part of it."

3: Liverpool is the best soccer team in the Premier League," he said.

"It's a great club and I'm very proud to be a part of it and
4: Liverpool is the best soccer team in the Premier League," he said.

"It's a great club and I'm very proud to be part of it. I


**Now that's much better! The 5 different beam hypotheses are pretty much all the same, but if we increaed `num_beams`, then we would see some more variation in the separate beams. But of course, Beam Search is not perfect either. It works well when the legnth of the generated text is more or less constant, like problems in translation or summarization, but not so much for open-ended problems like dialog or story generation (because it is much harder to find a balance between `num_beams` and `no_repeat_ngram_size`)**

**Furthermore, [research](https://arxiv.org/abs/1904.09751) shows that human languages do not follow this 'high probability word next' distribution. This makes sense - if my words were exactly what you expected them to be, I would be quite a boring person and most people don't want to be boring! The below graph plots the difference of Beam Search and actual human speech: ![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)**

Taken from original paper [here](https://arxiv.org/abs/1904.09751)

## Basic Sampling

**Now we will explore indeterministic decodings - sampling. Instead of following a strict path to find the end text with the highest probability, we instead randomly pick the next word by its conditional probability distribution:**

$$w_t \sim P(w|w_{1:t-1})$$

**However, when we include this randomness, the generated text tends to be incoherent (see more [here](https://arxiv.org/pdf/1904.09751.pdf)) so we can include the `temperature` parameter which increases the chances of high probability words and decreases the chances of low probability words in the sampling:**

**We just need to set `do_sample = True` to implement sampling and for demonstration purposes (you'll shortly see why) we set `top_k = 0`:**

In [8]:
# use temperature to decrease the sensitivity to low probability candidates
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_k = 0, 
                             temperature = 0.8
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
Liverpool is the best soccer team in the Premier League

"After spending so much time talking to fans, I can see a real sense of pride for the club and


## Top-K Sampling

**In Top-K sampling, the top k most likely next words are selected and the entire probability mass is shifted to these k words. So instead of increasing the chances of high probability words occuring and decreasing the chances of low probabillity words, we just remove low probability words all together**

**We just need to set `top_k` to however many of the top words we want to consider for our conditional probability distribution:**

In [9]:
#sample from only top_k most likely words
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_k = 50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
Liverpool is the best soccer team in the Premier League right now, it is going to be hard to beat them in the future, and I think the fans are looking towards ...


**Top-K Sampling seems to generate more coherent text than our random sampling before. But we can do even better:**

## Top-P Sampling

**Top-P sampling (also known as nucleus sampling) is similar to Top-K, but instead of choosing the top k most likely wordsm we choose the smallest set of words whose total probability is larger than p, and then the entire probability mass is shifted to the words in this set**

**The main difference here is that with Top-K sampling, the size of the set of words is static (obviously) whereas in Top-P sampling, the size of the set can change. To use this sampling method, we just set `top_k = 0` and choose a value `top_p`:**

In [10]:
#sample only from 80% most likely words
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_p = 0.8, 
                             top_k = 0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
Liverpool is the best soccer team in the Premier League, and they can play in Europe at any time."

Dennis, who is leading the effort to save the ...


## Top-K and Top-P Sampling

**As you could have probably guessed, we can use both Top-K and Top-P sampling here. This reduces the chances of us getting weird words (low probability words) while allowing for a dynamic selection size. We need only top a value for both `top_k` and `top_p`. We can even include the inital temperature parameter if we want to, Let's now see how our model performs now after adding everything together. We will check the top 5 return to see how diverse our answers are:**

In [11]:
#combine both sampling techniques
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = 2*MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .7,
                              top_k = 50, 
                              top_p = 0.85, 
                              num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: Liverpool is the best soccer team in the Premier League, period. If there was a poll, they'd win. They're the most popular team and the most popular team in the country, period."

That's not to say the Red Bulls have a ton of fans. In fact, they're in the bottom third of the league...

1: Liverpool is the best soccer team in the Premier League right now, but we need to go into it knowing that the club is also a great business and we have to get the right players," said Klopp, who won the Premier League title with Liverpool in 2013 and then guided the Reds to the Champions League final in 2016.

"So...

2: Liverpool is the best soccer team in the Premier League."

The striker scored a spectacular volley for City against Watford in the League Cup at the weekend.

He will now be hoping to inspire Liverpool's first win of the Premier League season on Saturday.

City have won thei

# III. Benchmark Prompts

**Here, we will see how well the GPT-2 model does when given some more interesting inputs. The following prompts are taken from [OpenAI's GPT2](https://openai.com/blog/better-language-models/) website where they feed them to their full sized GPT2 model (and the results are astounding, I highly recommend you check out their [page](https://openai.com/)**

In [12]:
MAX_LEN = 100

## "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English."

In [13]:
prompt1 = 'In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.'

input_ids = tokenizer.encode(prompt1, return_tensors='tf')

In [14]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85 
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. The new findings could open up a whole new world of unicorns and could help scientists understand what makes them unique and special.

"It's so exciting to see a species in the wild who speaks so perfectly," said Dr. David Smith, a graduate student in wildlife...



## "Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today."

**Can we use GPT-2 to generate fake news stories?**

In [15]:
prompt2 = 'Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today.'

input_ids = tokenizer.encode(prompt2, return_tensors='tf')

In [16]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today. The video of the incident was posted online. (Photo by Matt Sayles/Invision/AP)

The mother of 'Glee' singer Miley Cyrus said she was "devastated" by the theft of a $250 ring she was wearing on her finger.

"My son's a little girl. He's got an 8-year-old sister and she knows...



## "Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry."

**Can we use GPT-2 to imagine alternate histories of Lord of the Rings?**

In [17]:
prompt3 = 'Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.'

input_ids = tokenizer.encode(prompt3, return_tensors='tf')

In [18]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85 
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. Before the two could strike at their foe, however, a blast of fire from a great tree overhead threw the warriors back. As the trees collapsed, the orcs fell to the ground. They were all dead.

Roland was distraught. The orcs were a threat to the safety of the kingdom, and he needed to get rid of them as quickly as possible. After a long moment of thought,...



## "For today’s homework assignment, please describe the reasons for the US Civil War."

**Can we use GPT-2 to do our homework?**

In [19]:
prompt4 = "For today’s homework assignment, please describe the reasons for the US Civil War."

input_ids = tokenizer.encode(prompt4, return_tensors='tf')

In [20]:
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .8,
                              top_k = 50, 
                              top_p = 0.85 
                              #num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: For today’s homework assignment, please describe the reasons for the US Civil War. (It doesn't matter if you're a Civil War historian, historian, historian-fiction writer, or someone with a love for American history, we're looking for reasons for the war. Not your own reasons. Just the reasons.)

The US Civil War was a war that changed America. In addition to taking place in the US, it happened in the British Isles. There was a battle near the...



**Now we can see why OpenAI is hesitate to release their full scale model (recall, over 1.5 billion parameters) to the general public...it has the potential to do a lot of harm when used with malevolent intentions. Luckily, they have still provided us smaller versions that allow us to demonstrate just how powerful Transformer models are when applied to natural language and how they provide an analytical space from which to study linguistics.**

**Feel free to fork this notebook and experiment with your own inputs and model parameters to see what type of computer generated stories you can create!**

# References

**Below are the only references you need to implement your own GPT-2 model for text generation**


> https://arxiv.org/abs/1904.09751  
> https://openai.com/blog/better-language-models/  
> https://huggingface.co/transformers/model_doc/gpt2.html  
> https://huggingface.co/blog/how-to-generate  